# Robotics and Machine Vision – Image Warping Lab

Last edited by Nadejda Roubtsova for September - October 2024 

This lab will explore image warping.

You should download the **supporting images** `mona.jpg` and `windows.jpg` from the assignment's webpage, and put them into the same directory as this notebook.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
## Make figures larger.
plt.rcParams['figure.figsize'] = [10, 6]

## Try commenting this out if plots look blurry on your screen.
# %config InlineBackend.figure_format = 'retina'


## 1. Forward mapping [15 marks]

Let us start by implementing image warping using forward mapping.
Every pixel in the source image is transformed to the target image independently. You will see that this approach leads to gaps between the transformed pixels in the target image.



Let's first load the image and define an example warping transformation for this assignment:

In [ ]:
source = plt.imread('mona.jpg') / 255.

## Basic transformations to manipulate the source image.
T = np.array([[1, 0, -source.shape[1] / 2],
              [0, 1, -source.shape[0] / 2],
              [0, 0, 1]])
t = np.pi / 4
R = np.array([[np.cos(t), -np.sin(t), 0],
              [np.sin(t),  np.cos(t), 0],
              [ 0, 0, 1]])
S = np.diag([2, 2, 1])

## The warping transformation (rotation about arbitrary point).
M = np.linalg.inv(T) @ R @ S @ T

<span style="color:red"> **For you to do:** </span>

**a) Implement a function to transform a point (x, y) using a homogeneous 2D transform matrix, rounding to the nearest pixel. [5 marks]**

In [ ]:
def transform_pixel_nn(x, y, transform):
    """Transforms a source pixel coordinate (x, y) using 'transform', and rounds to the nearest pixel
    coordinate. Returns a tuple (x', y')."""
    ##  Replace this code with your own implementation.
    return (0, 0)

<span style="color:red"> **For you to do:** </span>

**b) Implement forward mapping using your per-pixel transform function. The pixel grid is meant to be kept consistent between the source and target. [10 marks]**

In [ ]:
def forward_mapping(source, transform):
    """Warps the 'source' image by the given 'transform' using forward mapping."""
    ## Replace this code with your own implementation.
    return np.zeros_like(source)


## Visualise input (left) and warped output (right).
target = forward_mapping(source, M)
plt.imshow(np.hstack([source, target]))
plt.axis("off");


## 2. Backward mapping [20 marks]

<span style="color:red"> **For you to do:** </span>

**a) Using the `transform_pixel_nn` function from your implementation of forward mapping implement backward mapping in the code cell below using the same 2D transformation `M`.**

Backward mapping computes for each **target image pixel** where in the source image it originats from, and, in its simplest form, uses the colour sampled from the nearest pixel. Note that this operation should not leave any gaps in the warped image.

In [ ]:
def backward_mapping(source, transform):
    """Warps the 'source' image by the given 'transform' using backward mapping with nearest-neighbour interpolation."""
    ## Replace this code with your own implementation.
    return np.zeros_like(source)


## Visualise input (left) and warped output (right).
target = backward_mapping(source, M)
plt.imshow(np.hstack([source, target]))
plt.axis("off");

## 3. Linear interpolation [20 marks]

<span style="color:red"> **For you to do:** </span>

**a) In the code cell below, modify your backward mapping function from part 2 to sample pixel colours from the source image using bilinear interpolation. [15 marks]**

**b) Handle the edge cases carefully for full marks. Specifically, implement the "fading to black" effect at the edges of the interpolated image by carefully considering how to handle the off-the-image grid pixels of the source image during interpolation. [5 marks]**

**Hint:** For validation, it may be helpful to use the transformations `M` commented out in the code cell below.

Interpolation must be implemented from scratch, not using any exisiting library functions. Note that interpolation may require a different precision of mapping and/or rounding approach to anchor to the pixel grid. So, in this question, feel free **not** to use `transform_pixel_nn` from Q1 a), which rounds off to the nearest pixel, or to write a *modified* version of the function.

In [ ]:
def backward_mapping_bilinear(source, transform):
    """Warps the 'source' image by the given 'transform' using backward mapping with bilinear interpolation."""
    ## Replace this code with your own implementation to satify 3a and 3b
    return np.zeros_like(source)


## Compare nearest-neighbour (left) and bilinear interpolation (right) side by side.
M = np.array([[12, 0, -2486], [0, 12, -2508], [0, 0, 1]])  # big smile
# M = np.array([[40, 0, 80], [0, 40, 80], [0, 0, 1]])  # check edge handling
target_nearest  = backward_mapping(source, M)
target_bilinear = backward_mapping_bilinear(source, M)
plt.imshow(np.hstack([target_nearest, target_bilinear]))
plt.axis("off");


## 4. Lens undistortion [45 marks]

In this part of the assignment, you will apply a different warping function, speficifically one that can be used to remove lens distortion from images. The visual effect of such *un-*distortion is rectification of lines so that they appear straight again.

Undistortion is to be implemented via backward mapping. To this end, consider the steps to map a target image location $(u,v)$ onto the source image location $(u', v')$ using the *polynomial lens distortion model*:

$$\begin{align}
x &= (u - p_x) / f_x \\
y &= (v - p_y) / f_y \\
r^2 &= x^2 + y^2 \\
x' &= x \cdot (1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
y' &= y \cdot (1 + k_1 r^2 + k_2 r^4 + k_3 r^6) \\
u' &= x' \cdot f_x + p_x \\
v' &= y' \cdot f_y + p_y
\end{align}$$

In the mapping above:
* $f_x$ and $f_y$ are the focal lengths of the camera
* $(p_x, p_y)$ is called the *principal point* or *centre of projection*
* $k_1$, $k_2$, $k_3$ are the *lens distortion coefficients*

Further please note that:
* The `camera_matrix` is defined as:

  $\mathbf{K} = \begin{bmatrix}f_x & 0 & p_x \\ 0 & f_y & p_y \\ 0 & 0 & 1\end{bmatrix}$.
  

* `dist_coeffs` is the lens distortion coefficients $\begin{bmatrix}k_1 & k_2 & k_3\end{bmatrix}$ .

<span style="color:red"> **For you to do:** </span>

**a) Implement the steps of the polynomial distortion model detailed above in the function below to calculate the source image location $(u', v')$  for a single given pixel $(u,v)$ of the target (undistorted) image. [5 marks]**



In [1]:
def undistort_point(u, v, camera_matrix, dist_coeffs):
    """Given camera matrix and distortion coefficients, finds the corresponding undistorted (source) pixel location (u', v') 
    for a given (target) pixel location (u,v)."""
    ## Replace this code with your own implementation.
    return 0, 0

<span style="color:red"> **For you to do:** </span>

**b) Using your function `undistort_point` above, in the code cell below implement polynomial lens undistortion for a given image via backward mapping with bilinear interpolation. [15 marks]**


In [ ]:
def undistort_image(image, camera_matrix, dist_coeffs):
    """Undistorts an image using the given camera matrix and distortion coefficients."""
    ## Replace this code with your own implementation.
    return np.zeros_like(image)


## The intrinsic camera matrix and lens undistortion coefficients for the test image.
source = plt.imread('window.jpg') / 255.
camera_matrix = np.array([[474.53, 0, 405.96], [0, 474.53, 217.81], [0, 0, 1]])
dist_coeffs = np.array([-0.27194, 0.11517, -0.029859])

## Visualise input (top) and undistorted output (bottom).
target_unvectorised = undistort_image(source, camera_matrix, dist_coeffs)
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(np.vstack([source, target_unvectorised]))
plt.axis("off");

<span style="color:red"> **For you to do:** </span>

**c) Implement fast image undistortion using vectorisation, without any for-loops. Your function should run in less than 3 seconds. [15 marks]**

Remember that the vectorised version of your undistortion function must be functionally identical to the original. This means the function must also perform bilinear interpolation.

This is the advanced question of the assignment, so it will be marked stricter than 4b in terms of correctness of output and performance, with partial credit awarded only rarely. Watch out for hidden for-loops in your implementation.

In [ ]:
def undistort_image_vectorised(image, camera_matrix, dist_coeffs):
    """Undistorts an image using the given camera matrix and distortion coefficients.
    Use vectorised operations to avoid slow for-loops."""
    ## Replace this code with your own implementation.
    return np.zeros_like(image)


# Visualise input (top) and undistorted output (bottom).
target_vectorised = undistort_image_vectorised(source, camera_matrix, dist_coeffs)
plt.rcParams['figure.figsize'] = [10, 10]
plt.imshow(np.vstack([source, target_vectorised]))
plt.axis("off");

<span style="color:red"> **For you to do:** </span>

**d) Show that both the original and the vectorised implementations produce the same result by running the cell below. [10 marks]**

Partial credit will be awarded if the outputs are consistent with each other but not completely correct. If the output is completely wrong given the task (e.g., not undistorted), no credit will be awarded.

Note that for consistent outputs the difference below will be very small (say, order of e-15) but not completely zero.

If you find a discrepancy between the outputs, go back and debug the relevant implementation.

In [ ]:
difference = np.sum(np.abs(target_unvectorised - target_vectorised))
print(difference)